In [7]:
import pandas as pd

import streamlit as st
import pandas as pd
import os
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive


In [8]:
# Load Data
article = pd.read_csv("articles.csv")
district = pd.read_csv("District_beneficiary.csv")
public = pd.read_excel("Public_Beneficiary_21_24.xlsx")



article_name = article["Articles"].unique().tolist()

if article_name:
    for art in article_name:

        cpu = article[article["Articles"] == art]["Cost per unit"].tolist()[0]
        print(cpu)


100000.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
5000.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
10000.0
20500.0
20500.0
20500.0
20500.0
5000.0


In [9]:
article

,Articles,Cost per unit,Item Type
0,Handicapped Hand Tricycle,100000.0,Article
1,Fixed Wheel Chair,20500.0,Article
2,Wheelchair,20500.0,Article
3,Handicapped Scooter,20500.0,Article
4,Two Wheeler,20500.0,Article
...,...,...,...
65,Sewing Machine Heavy,20500.0,Article
66,Sewing Machine ORD / Motor,20500.0,Article
67,Sewing Machine ORD,20500.0,Article
68,Gents Cycle,20500.0,Article


In [10]:
article

,Articles,Cost per unit,Item Type
0,Handicapped Hand Tricycle,100000.0,Article
1,Fixed Wheel Chair,20500.0,Article
2,Wheelchair,20500.0,Article
3,Handicapped Scooter,20500.0,Article
4,Two Wheeler,20500.0,Article
...,...,...,...
65,Sewing Machine Heavy,20500.0,Article
66,Sewing Machine ORD / Motor,20500.0,Article
67,Sewing Machine ORD,20500.0,Article
68,Gents Cycle,20500.0,Article


In [2]:
import io
import json
import streamlit as st
import pandas as pd
import os
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from googleapiclient.http import MediaFileUpload, MediaIoBaseUpload, MediaIoBaseDownload
from googleapiclient.discovery import build
from google.oauth2 import service_account
import streamlit_authenticator as stauth
import yaml
import time
from yaml.loader import SafeLoader

master_data_id = "1ry614-7R4-s0uQcv0zrNeS4O0KAbhVEC67rl5_VllGI"  # Replace with your file's ID
article_data_id = "1b7eyqlN3lTapBRYcO1VrXGsj_gBVSxQLIyLCPu3UcG8"
district_data_id = "1lwJL-_KQaOY3VSd2cOeOdiR5QOn8yvX3zp6xNfQJo9U"
public_data_id = "1sO08BfwN1gzNs_N7XDq1RnqMgJDDKMdq_nsaNhmjKhs"

## Access API from local json
creds = service_account.Credentials.from_service_account_file('mnpdatabase-ca1a93fefdd6.json',
        scopes=['https://www.googleapis.com/auth/drive'])

## Access via streamlit secrets
credentials_dict = json.loads(st.secrets["gcp"]["credentials"])
# creds = service_account.Credentials.from_service_account_info(credentials_dict, scopes=['https://www.googleapis.com/auth/drive'])
drive_service = build('drive', 'v3', credentials=creds)


2025-01-25 18:01:50.234 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-25 18:01:50.238 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-25 18:01:50.241 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-25 18:01:50.244 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [3]:
def read_file(file_id):
    try:
        # Specify the desired MIME type for export (e.g., text/csv for Sheets)
        request = drive_service.files().export_media(
            fileId=file_id,
            mimeType='text/csv'  # Change this based on the file type
        )
        file_stream = io.BytesIO()
        downloader = MediaIoBaseDownload(file_stream, request)

        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download progress: {int(status.progress() * 100)}%")

        file_stream.seek(0)  # Reset the stream position
        df = pd.read_csv(file_stream)
        return df

    except Exception as e:
        st.error(f"Failed to read file: {e}")

In [4]:


def update_file(file_id, updated_df):
    updated_stream = io.BytesIO()
    updated_df.to_csv(updated_stream, index=False)
    updated_stream.seek(0)

    media = MediaIoBaseUpload(updated_stream, mimetype="text/csv")
    updated_file = drive_service.files().update(
        fileId=file_id,
        media_body=media).execute()

    alert1 = st.success(f"File updated: {updated_file.get('id')}")
    time.sleep(1)
    alert1.empty()

In [5]:
a = ["P 001", "P 002", "P 003", "P 004", "P555", "P 006"]
ab = int(a[0].split(" ")[1])
f"P {ab+1:03d}"

'P 002'

In [19]:
public_master_id = "1EdEySmYe6ZJUW16f65_q30nkqfbvDADjcmEkAEJrrL4"

public = read_file(public_data_id)
public_master = read_file(public_master_id)
public_master

Download progress: 100%
Download progress: 100%


,App. No.,Name,Aadhar (Without Space),Handicapped (Yes / No),Address,Mobile,Article Name,Comments
0,P 001,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
app_no = "P 001"
if app_no in public_master["App. No."].values:
    print("Application Number already exists in the database.")

Application Number already exists in the database.


In [18]:

public_master["Aadhar (Without Space)"]

Series([], Name: Aadhar (Without Space), dtype: object)